In [1]:
import pandas as pd
import numpy as np

## **1. Load and Merge the Datasets**
We load two datasets:
- **Application Record**: Contains applicant details (income, occupation, education, etc.).
- **Credit Record**: Contains their credit status and history.

After loading, we merge them based on the **"ID"** column.


In [7]:
# Path for initial datasets
application_record_path = r'C:\Users\vagel\Desktop\CR_Risk Project\Dataset\application_record.csv'
credit_record_path = r'C:\Users\vagel\Desktop\CR_Risk Project\Dataset\credit_record.csv'

# Load the datasets
application_record = pd.read_csv(application_record_path)
credit_record = pd.read_csv(credit_record_path)

# Preview the datasets
display(application_record)
display(credit_record)


,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438552,6840104,M,N,Y,0,135000.0,Pensioner,Secondary / secondary special,Separated,House / apartment,-22717,365243,1,0,0,0,NaN,1
438553,6840222,F,N,N,0,103500.0,Working,Secondary / secondary special,Single / not married,House / apartment,-15939,-3007,1,0,0,0,Laborers,1
438554,6841878,F,N,N,0,54000.0,Commercial associate,Higher education,Single / not married,With parents,-8169,-372,1,1,0,0,Sales staff,1
438555,6842765,F,N,Y,0,72000.0,Pensioner,Secondary / secondary special,Married,House / apartment,-21673,365243,1,0,0,0,NaN,2


,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C
...,...,...,...
1048570,5150487,-25,C
1048571,5150487,-26,C
1048572,5150487,-27,C
1048573,5150487,-28,C


### **1.1 Initial Pre-Processing: Rename Columns for Better Readability**

The dataset contains generic column names. We rename them to more descriptive names to improve clarity and ease of use.


In [8]:
# Rename columns for better readability in application_record dataset
renamed_columns_app = {
    "ID": "Applicant_ID",
    "CODE_GENDER": "Gender",
    "FLAG_OWN_CAR": "Owns_Car",
    "FLAG_OWN_REALTY": "Owns_Property",
    "CNT_CHILDREN": "Number_of_Children",
    "AMT_INCOME_TOTAL": "Income_Total",
    "NAME_INCOME_TYPE": "Income_Type",
    "NAME_EDUCATION_TYPE": "Education_Level",
    "NAME_FAMILY_STATUS": "Family_Status",
    "NAME_HOUSING_TYPE": "Housing_Type",
    "DAYS_BIRTH": "Age_in_Days",
    "DAYS_EMPLOYED": "Days_Employed",
    "FLAG_MOBIL": "Has_Mobile_Phone",
    "FLAG_WORK_PHONE": "Has_Work_Phone",
    "FLAG_PHONE": "Has_Phone",
    "FLAG_EMAIL": "Has_Email",
    "OCCUPATION_TYPE": "Occupation_Type",
    "CNT_FAM_MEMBERS": "Family_Members_Count"
}

# Apply new column names to application_record
application_record.rename(columns=renamed_columns_app, inplace=True)

# Rename columns for better readability in credit_record
renamed_columns_credit = {
    "ID": "Applicant_ID",
    "MONTHS_BALANCE": "Months_Balance",
    "STATUS": "Credit_Status"
}

# Apply new column names to credit_record
credit_record.rename(columns=renamed_columns_credit, inplace=True)

### **1.2 Map Credit Status to Binary Values**

The **credit status column** has multiple categories. We simplify it into a **binary system**:
- **0** → No overdue payments
- **X** → No loan for the month
- **C** → Account closed
- **1** → Any overdue payments
- **2,3,4,5** → Any overdue payments in the corresponding month

This process allows us to create our target variable : **"Credit_Status"**

In [9]:
# Replace "C" and "X" with "0" to treat them as non-default
credit_record["Credit_Status"] = credit_record["Credit_Status"].replace({"C": "0", "X": "0"})
credit_record["Credit_Status"] = credit_record["Credit_Status"].astype(int)  # Convert to numeric

# Aggregate to get the worst (maximum) status per Applicant_ID
credit_record_agg = credit_record.groupby("Applicant_ID").agg(
    worst_status=("Credit_Status", "max")  # Maximum recorded status per Applicant_ID
).reset_index()

# Merge with application data
merged_data = application_record.merge(credit_record_agg, on="Applicant_ID", how="inner")

# Define the default target (1 = default, 0 = non-default)
merged_data["Credit_Status"] = merged_data["worst_status"].apply(lambda x: 1 if x >= 1 else 0)

### **1.3 Identify the missing value for each variable**

Some columns might contain missing values like  **Occupation_Type** column.

In [11]:
# Check for missing values in each column
missing_values = merged_data.isnull().sum().reset_index()
missing_values.columns = ["Variable", "Missing_Values_Count"]
missing_values["Percentage_Missing"] = (missing_values["Missing_Values_Count"] / len(merged_data)) * 100

missing_values

,Variable,Missing_Values_Count,Percentage_Missing
0,Applicant_ID,0,0.000000
1,Gender,0,0.000000
2,Owns_Car,0,0.000000
3,Owns_Property,0,0.000000
4,Number_of_Children,0,0.000000
5,Income_Total,0,0.000000
6,Income_Type,0,0.000000
7,Education_Level,0,0.000000
8,Family_Status,0,0.000000
9,Housing_Type,0,0.000000


### **1.4 Convert Days to Years for Better Interpretation**

Some columns (age, employment duration) are given in **days**, which is not intuitive. We convert them to **years**.

In [12]:
# Convert "Age_in_Days" and "Days_Employed" to years
merged_data["Age_in_Days"] = (merged_data["Age_in_Days"] / -365).round(1)
merged_data["Days_Employed"] = (merged_data["Days_Employed"] / -365).round(1)

# Rename the columns to reflect the change
merged_data.rename(columns={
    "Age_in_Days": "Age_in_Years",
    "Days_Employed": "Years_Employed"
}, inplace=True)

In [13]:
# Count the number of defaults and non-defaults
default_counts = merged_data["Credit_Status"].value_counts()

default_counts

Credit_Status
0    32166
1     4291
Name: count, dtype: int64

### **1.5 Impute Missing Values in "Occupation_Type" Using a Probabilistic Approach**
Many applicants have **missing occupation data**.

1. **Calculate the probability** of each occupation type for people with the same **education and income type**.

2. **Randomly assign** a missing occupation **based on these probabilities**.


In [63]:
# Compute probabilities of each occupation given Education and Income Type
occupation_probs = merged_data.dropna(subset=["Occupation_Type"]).groupby(["Education_Level", "Income_Type"])["Occupation_Type"].value_counts(normalize=True).unstack()

# Function to impute missing occupation values
def impute_occupation(row):
    if pd.isna(row["Occupation_Type"]):
        edu = row["Education_Level"]
        inc = row["Income_Type"]
        if edu in occupation_probs.index and inc in occupation_probs.columns:
            probabilities = occupation_probs.loc[edu, :].dropna()
            if not probabilities.empty:
                return np.random.choice(probabilities.index, p=probabilities.values)
    return row["Occupation_Type"]

# Apply the function to fill missing occupation values
merged_data["Occupation_Type"] = merged_data.apply(impute_occupation, axis=1)


### **1.6 Handle Any Remaining Missing Occupations**
Some cases might still have missing values (e.g., if no matching education-income type exists). We fill these with the **most common occupation**.


In [64]:
# Check remaining missing values in "Occupation_Type"
missing_occupations = merged_data["Occupation_Type"].isna().sum()

# If any are still missing, fill with the most common occupation
if missing_occupations > 0:
    most_common_occupation = merged_data["Occupation_Type"].mode()[0]
    merged_data["Occupation_Type"].fillna(most_common_occupation, inplace=True)


### **1.7 Save the Cleaned Dataset**
Finally, we save the cleaned dataset for further analysis and modeling.


In [14]:
# Save the modified dataset to a new CSV file
analysis_ready_path = r'C:/Users/vagel/Desktop/C.R/credit_risk_analysis.csv'
merged_data.to_csv(analysis_ready_path, index=False)

merged_data.head()


,Applicant_ID,Gender,Owns_Car,Owns_Property,Number_of_Children,Income_Total,Income_Type,Education_Level,Family_Status,Housing_Type,Age_in_Years,Years_Employed,Has_Mobile_Phone,Has_Work_Phone,Has_Phone,Has_Email,Occupation_Type,Family_Members_Count,worst_status,Credit_Status
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,32.9,12.4,1,1,0,0,NaN,2,1,1
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,32.9,12.4,1,1,0,0,NaN,2,1,1
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,58.8,3.1,1,0,0,0,Security staff,2,0,0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,52.4,8.4,1,0,1,1,Sales staff,1,0,0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,52.4,8.4,1,0,1,1,Sales staff,1,0,0
